In [1]:
import pandas as pd
df = pd.read_excel("Flow_depth.xls")
df = df.iloc[2:].reset_index(drop=True)
df = df.rename(columns = {'IDs:':'Date/Time'})
df.head()

,Date/Time,92090040,92090041,92090042,92090070,92090090,92090100,92100100,92100110,92100120,...,92100190,92100220,92100230,92100240,92100250,92100260,92100280,92100300,92100320,OF-1
0,2023-10-01 00:10:00,0.124174,0.06161,0.065682,0.070087,0.095688,0.058608,0.062742,0.06277,0.058495,...,0.024923,0.078109,0.124467,0.091907,0.064839,0.039564,0.12718,0.102981,0.116926,0.048051
1,2023-10-01 00:20:00,0.195642,0.061608,0.126653,0.081124,0.144584,0.128025,0.057909,0.094791,0.057865,...,0.03182,0.155917,0.17709,0.141373,0.146733,0.100651,0.140623,0.162597,0.117399,0.08378
2,2023-10-01 00:30:00,0.264221,0.061608,0.183327,0.131465,0.200239,0.191842,0.057831,0.108786,0.066044,...,0.032719,0.198622,0.216264,0.17466,0.193832,0.150146,0.180615,0.171752,0.117279,0.11803
3,2023-10-01 00:40:00,0.331028,0.061608,0.242019,0.173434,0.242876,0.233405,0.05783,0.108009,0.065615,...,0.032861,0.233713,0.248647,0.203819,0.231262,0.166831,0.200379,0.163299,0.117266,0.151517
4,2023-10-01 00:50:00,0.380019,0.061608,0.287735,0.201139,0.270772,0.257954,0.05783,0.107841,0.065494,...,0.032884,0.243812,0.260042,0.209928,0.24409,0.165327,0.201782,0.159117,0.117265,0.176059


In [2]:
df.shape

(17706, 24)

In [3]:
import pandas as pd
import numpy as np

# --- assume df is your DataFrame and the datetime column is named 'Date/Time' ---
# If your datetime column has a different name, change it below.
df = df.copy()
df['Date/Time'] = pd.to_datetime(df['Date/Time'])

# create year/month/half indicator (half = 1 for 1..15, 2 for 16..end)
df['year']  = df['Date/Time'].dt.year
df['month'] = df['Date/Time'].dt.month
df['half']  = np.where(df['Date/Time'].dt.day <= 15, 1, 2)

# assign a sequential half-month id (1,2,3...) in chronological order
# groupby(..., sort=True).ngroup() will produce 0-based group numbers in sorted order
df['period_id'] = df.groupby(['year','month','half'], sort=True).ngroup() + 1
df['period'] = 't' + df['period_id'].astype(str)

# choose the node columns to aggregate (everything except helper columns)
exclude = ['Date/Time','year','month','half','period_id','period']
node_cols = [c for c in df.columns if c not in exclude]

# --- AGGREGATION OPTION A: SUM (default) ---
agg_sum = df.groupby('period', sort=True)[node_cols].sum().reset_index()

# If you want a meaningful representative datetime for each period, add:
# create a mapping from period -> (period_start, period_end) using the earliest/largest timestamp
period_bounds = (df.groupby('period', sort=True)
                   .agg(period_start=('Date/Time','min'),
                        period_end  =('Date/Time','max'))
                   .reset_index())
agg_sum = agg_sum.merge(period_bounds, on='period')

# --- AGGREGATION OPTION B: MEAN (if you prefer averages instead of sums) ---
agg_mean = df.groupby('period', sort=True)[node_cols].mean().reset_index()
agg_mean = agg_mean.merge(period_bounds, on='period')

agg_sum.head()

,period,92090040,92090041,92090042,92090070,92090090,92090100,92100100,92100110,92100120,...,92100230,92100240,92100250,92100260,92100280,92100300,92100320,OF-1,period_start,period_end
0,t1,1118.366614,362.131813,937.115035,742.965968,902.588018,880.567281,224.429246,435.369393,360.795,...,883.774382,800.516804,848.711348,730.302622,822.668796,624.656123,453.080248,477.663732,2023-10-01 00:10:00,2023-10-15 23:50:00
1,t2,993.286148,187.46714,771.934796,519.433318,679.934711,629.877073,140.397565,260.997201,162.731102,...,623.48268,500.838152,586.41944,392.550871,480.509519,380.002897,281.268429,463.277945,2023-10-16 00:00:00,2023-10-31 23:50:00
2,t3,1071.850851,312.684667,880.229153,640.655564,791.121892,733.807076,148.62942,291.922287,198.491259,...,723.095552,597.385651,681.993819,467.596833,564.242461,400.016816,286.631607,479.87295,2023-11-01 00:00:00,2023-11-15 23:50:00
3,t4,911.484093,157.81529,703.70186,473.115994,623.57605,577.509354,128.02156,238.552254,146.817318,...,571.766781,458.83437,537.526573,359.66011,440.492768,348.496357,259.034074,425.032914,2023-11-16 00:00:00,2023-11-30 23:50:00
4,t5,1081.385173,327.57501,898.984968,741.854482,892.255686,912.61288,325.47274,547.218435,454.718202,...,931.758095,861.730544,909.265801,794.105027,881.080226,703.442691,538.339509,454.997627,2023-12-01 00:00:00,2023-12-15 23:50:00


In [4]:
agg_sum.to_csv("Flow_depth.csv",index=False)

In [5]:
agg_sum.shape

(8, 26)

In [6]:
df = pd.read_excel("Flow_rate.xlsx")
df = df.iloc[2:].reset_index(drop=True)
df = df.rename(columns = {'IDs:':'Date/Time'})
df.head()

,Date/Time,92090040,92090041,92090042,92090070,92090090,92090100,92100100,92100110,92100120,...,92100190,92100220,92100230,92100240,92100250,92100260,92100280,92100300,92100320,OF-1
0,2023-10-01 00:10:00,0.029362,0.012352,0.017731,0.010798,0.030601,0.022105,0.006998,0.010816,0.007112,...,0.002225,0.025429,0.024921,0.033699,0.013563,0.014226,0.031361,0.051982,0.030312,0.01432
1,2023-10-01 00:20:00,0.055978,0.012352,0.04493,0.010798,0.059008,0.057283,0.006998,0.01969,0.007112,...,0.002136,0.06274,0.06432,0.05087,0.054391,0.042528,0.050464,0.055739,0.030312,0.040238
2,2023-10-01 00:30:00,0.095967,0.012352,0.086999,0.010798,0.105134,0.098656,0.006998,0.025159,0.007112,...,0.002134,0.097285,0.100896,0.077127,0.089243,0.06186,0.078598,0.055672,0.030312,0.077798
3,2023-10-01 00:40:00,0.142564,0.012352,0.134896,0.010798,0.14484,0.134688,0.006998,0.02557,0.007112,...,0.002134,0.126184,0.136423,0.095605,0.123111,0.063293,0.090512,0.055665,0.030312,0.126097
4,2023-10-01 00:50:00,0.178755,0.012352,0.166028,0.010798,0.17138,0.152587,0.006998,0.025531,0.007112,...,0.002134,0.132118,0.149922,0.096855,0.134079,0.060209,0.089423,0.055664,0.030312,0.168424


In [7]:
import pandas as pd
import numpy as np

# --- assume df is your DataFrame and the datetime column is named 'Date/Time' ---
# If your datetime column has a different name, change it below.
df = df.copy()
df['Date/Time'] = pd.to_datetime(df['Date/Time'])

# create year/month/half indicator (half = 1 for 1..15, 2 for 16..end)
df['year']  = df['Date/Time'].dt.year
df['month'] = df['Date/Time'].dt.month
df['half']  = np.where(df['Date/Time'].dt.day <= 15, 1, 2)

# assign a sequential half-month id (1,2,3...) in chronological order
# groupby(..., sort=True).ngroup() will produce 0-based group numbers in sorted order
df['period_id'] = df.groupby(['year','month','half'], sort=True).ngroup() + 1
df['period'] = 't' + df['period_id'].astype(str)

# choose the node columns to aggregate (everything except helper columns)
exclude = ['Date/Time','year','month','half','period_id','period']
node_cols = [c for c in df.columns if c not in exclude]

# --- AGGREGATION OPTION A: SUM (default) ---
agg_sum = df.groupby('period', sort=True)[node_cols].sum().reset_index()

# If you want a meaningful representative datetime for each period, add:
# create a mapping from period -> (period_start, period_end) using the earliest/largest timestamp
period_bounds = (df.groupby('period', sort=True)
                   .agg(period_start=('Date/Time','min'),
                        period_end  =('Date/Time','max'))
                   .reset_index())
agg_sum = agg_sum.merge(period_bounds, on='period')

# --- AGGREGATION OPTION B: MEAN (if you prefer averages instead of sums) ---
agg_mean = df.groupby('period', sort=True)[node_cols].mean().reset_index()
agg_mean = agg_mean.merge(period_bounds, on='period')

agg_sum.head()

,period,92090040,92090041,92090042,92090070,92090090,92090100,92100100,92100110,92100120,...,92100230,92100240,92100250,92100260,92100280,92100300,92100320,OF-1,period_start,period_end
0,t1,610.882591,45.074791,540.511319,35.7061,536.09321,459.353793,27.003062,90.961834,39.323266,...,452.481335,293.383419,404.622396,185.803309,269.907138,174.850155,90.856822,610.15851,2023-10-01 00:10:00,2023-10-15 23:50:00
1,t2,506.320381,32.406693,453.104611,27.167593,443.807148,384.624917,18.277108,65.979911,18.190584,...,377.250529,240.365413,336.661717,150.881091,222.295095,142.265138,76.634723,506.319132,2023-10-16 00:00:00,2023-10-31 23:50:00
2,t3,605.032775,43.259344,535.952838,30.882656,524.291611,457.824422,24.036175,83.870007,22.131333,...,448.298503,288.539344,403.409959,183.129017,265.706974,172.29852,88.836276,605.039974,2023-11-01 00:00:00,2023-11-15 23:50:00
3,t4,452.961662,28.252345,406.240521,24.560499,398.003775,344.360106,15.994529,58.20853,16.210405,...,337.876183,214.80546,300.980195,134.503927,198.838601,126.88125,68.99367,452.962692,2023-11-16 00:00:00,2023-11-30 23:50:00
4,t5,552.115759,38.082181,490.610484,28.701432,479.676919,417.084613,21.261415,78.395713,20.276432,...,408.512211,258.519914,366.212703,161.556006,237.912549,152.577181,83.124575,552.212946,2023-12-01 00:00:00,2023-12-15 23:50:00


In [8]:
agg_sum.shape

(8, 26)

In [9]:
agg_sum.to_csv("Flow_rate.csv",index=False)